In [4]:
import matplotlib.animation as animation
from matplotlib import rc
from IPython.display import HTML, Image
from ipywidgets import interact,Dropdown,IntSlider,FloatRangeSlider, FloatSlider, RadioButtons
rc('animation', html='html5')

In [5]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from beakerx import *
plt.style.use('ggplot')
import seaborn as sns
sns.set()
import h5py
import os
from tqdm import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


# API
## DefectAugment
class gunpowder.DefectAugment(intensities, prob_missing=0.05, prob_low_contrast=0.05, prob_artifact=0.0, prob_deform=0.0, contrast_scale=0.1, artifact_source=None, artifacts=None, artifacts_mask=None, deformation_strength=20, axis=0)[source]

Augment intensity arrays section-wise with artifacts like missing sections, low-contrast sections, by blending in artifacts drawn from a separate source, or by deforming a section.

- intensities (ArrayKey) – The key of the array of intensities to modify.
- prob_missing (float) –
- prob_low_contrast (float) –
- prob_artifact (float) –
- prob_deform (float) – Probabilities of having a missing section, low-contrast section, an artifact (see param artifact_source) or a deformed slice. The sum should not exceed 1. Values in missing sections will be set to 0.
contrast_scale (float, optional) – By how much to scale the intensities for a low-contrast section, used if prob_low_contrast > 0.
- (class (artifact_source) – BatchProvider, optional):A gunpowder batch provider that delivers intensities (via ArrayKey artifacts) and an alpha mask (via ArrayKey artifacts_mask), used if prob_artifact > 0.

- artifacts (ArrayKey, optional) – The key to query artifact_source for to get the intensities of the artifacts.
- artifacts_mask (ArrayKey, optional) – The key to query artifact_source for to get the alpha mask of the artifacts to blend them with intensities.
- deformation_strength (int, optional) – Strength of the slice deformation in voxels, used if prob_deform > 0. The deformation models a fold by shifting the section contents towards a randomly oriented line in the section. The line itself will be drawn with a value of 0.
- axis (int, optional) – Along which axis sections are cut.


## ElasticAugment
class gunpowder.ElasticAugment(control_point_spacing, jitter_sigma, rotation_interval, prob_slip=0, prob_shift=0, max_misalign=0, subsample=1)[source]
Elasticly deform a batch. Requests larger batches upstream to avoid data loss due to rotation and jitter.

- control_point_spacing (tuple of int) – Distance between control points for the elastic deformation, in voxels per dimension.
- jitter_sigma (tuple of float) – Standard deviation of control point jitter distribution, in voxels per dimension.
rotation_interval (tuple of two floats) – Interval to randomly sample rotation angles from (0, 2PI).
- prob_slip (float) – Probability of a section to “slip”, i.e., be independently moved in x-y.
- prob_shift (float) – Probability of a section and all following sections to move in x-y.
- max_misalign (int) – Maximal voxels to shift in x and y. Samples will be drawn uniformly. Used if prob_slip + prob_shift > 0.
- subsample (int) – Instead of creating an elastic transformation on the full resolution, create one subsampled by the given factor, and linearly interpolate to obtain the full resolution transformation. This can significantly speed up this node, at the expense of having visible piecewise linear deformations for large factors. Usually, a factor of 4 can savely by used without noticable changes. However, the default is 1 (i.e., no subsampling).

## IntensityAugment
class gunpowder.IntensityAugment(array, scale_min, scale_max, shift_min, shift_max, z_section_wise=False)[source]
Randomly scale and shift the values of an intensity array.

- array (ArrayKey) – The intensity array to modify.
- scale_min (float) –
- scale_max (float) –
- shift_min (float) –
- shift_max (float) –
The min and max of the uniformly randomly drawn scaling and shifting values for the intensity augmentation. Intensities are changed as:
```
a = a.mean() + (a-a.mean())*scale + shift
```
- z_section_wise (bool) – Perform the augmentation z-section wise. Requires 3D arrays and assumes that z is the first dimension.

## SimpleAugment
class gunpowder.SimpleAugment(mirror_only=None, transpose_only=None)[source]
Randomly mirror and transpose all Arrays and Points in a batch.

Parameters:	
- mirror_only (list of int, optional) – If set, only mirror between the given axes. This is useful to exclude channels that have a set direction, like time.
- transpose_only (list of int, optional) – If set, only transpose between the given axes. This is useful to limit the transpose to axes with the same resolution or to exclude non-spatial dimensions.

In [1]:
import gunpowder

/home/chenxupeng/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
from __future__ import print_function

import math
import numpy as np
import random

from gunpowder import *
from gunpowder.caffe import *
from gunpowder.ext import malis

malis has some problems, don't use? doesn't matter in augmentation??

In [15]:
SolverParameters()

solver_parameters = SolverParameters()
solver_parameters.train_net = 'net.prototxt'
solver_parameters.base_lr = 1e-4
solver_parameters.momentum = 0.95
solver_parameters.momentum2 = 0.999
solver_parameters.delta = 1e-8
solver_parameters.weight_decay = 0.000005
solver_parameters.lr_policy = 'inv'
solver_parameters.gamma = 0.0001
solver_parameters.power = 0.75
solver_parameters.snapshot = 2000
solver_parameters.snapshot_prefix = 'net'
solver_parameters.type = 'Adam'
solver_parameters.resume_from = None
solver_parameters.train_state.add_stage('euclid')

import gunpowder as gp
gp.ElasticAugment(
        control_point_spacing=[4, 40, 40],
        jitter_sigma=[0, 2, 2],
        rotation_interval=[0, math.pi/2.0])

def train():

    #affinity_neighborhood = malis.mknhood3d()
    solver_parameters = SolverParameters()
    solver_parameters.train_net = 'net.prototxt'
    solver_parameters.base_lr = 1e-4
    solver_parameters.momentum = 0.95
    solver_parameters.momentum2 = 0.999
    solver_parameters.delta = 1e-8
    solver_parameters.weight_decay = 0.000005
    solver_parameters.lr_policy = 'inv'
    solver_parameters.gamma = 0.0001
    solver_parameters.power = 0.75
    solver_parameters.snapshot = 2000
    solver_parameters.snapshot_prefix = 'net'
    solver_parameters.type = 'Adam'
    solver_parameters.resume_from = None
    solver_parameters.train_state.add_stage('euclid')

    request = BatchRequest()
    request.add_array_request(ArrayKeys.RAW, (84,268,268))
    request.add_array_request(ArrayKeys.GT_LABELS, (56,56,56))
    request.add_array_request(ArrayKeys.GT_MASK, (56,56,56))
    request.add_array_request(ArrayKeys.GT_IGNORE, (56,56,56))
    request.add_array_request(ArrayKeys.GT_AFFINITIES, (56,56,56))

    data_sources = tuple(
        Hdf5Source(
            'sample_'+s+'_padded_20160501.aligned.filled.cropped.hdf',
            datasets = {
                ArrayKeys.RAW: 'volumes/raw',
                ArrayKeys.GT_LABELS: 'volumes/labels/neuron_ids_notransparency',
                ArrayKeys.GT_MASK: 'volumes/labels/mask',
            }
        ) +
        Normalize() +
        RandomLocation()
        for s in ['A', 'B', 'C']
    )

    artifact_source = (
        Hdf5Source(
            'sample_ABC_padded_20160501.defects.hdf',
            datasets = {
                ArrayKeys.RAW: 'defect_sections/raw',
                ArrayKeys.ALPHA_MASK: 'defect_sections/mask',
            }
        ) +
        RandomLocation(min_masked=0.05, mask_array_key=ArrayKeys.ALPHA_MASK) +
        Normalize() +
        IntensityAugment(0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0]) +
        SimpleAugment(transpose_only_xy=True)
    )

    snapshot_request = BatchRequest()
    snapshot_request.add_array_request(ArrayKeys.LOSS_GRADIENT, (56,56,56))

    batch_provider_tree = (
        data_sources +
        RandomProvider() +
        ExcludeLabels([8094], ignore_mask_erode=12) +
        ElasticAugment([4,40,40], [0,2,2], [0,math.pi/2.0], prob_slip=0.05,prob_shift=0.05,max_misalign=25) +
        SimpleAugment(transpose_only_xy=True) +
        GrowBoundary(steps=3, only_xy=True) +
        AddGtAffinities(affinity_neighborhood) +
        SplitAndRenumberSegmentationLabels() +
        IntensityAugment(0.9, 1.1, -0.1, 0.1, z_section_wise=True) +
        DefectAugment(
            prob_missing=0.03,
            prob_low_contrast=0.01,
            prob_artifact=0.03,
            artifact_source=artifact_source,
            contrast_scale=0.1) +
        ZeroOutConstSections() +
        IntensityScaleShift(2,-1) +
        BalanceAffinityLabels() +
        PreCache(
            cache_size=10,
            num_workers=5) +
        Train(solver_parameters, use_gpu=0) +
        Snapshot(every=10, output_filename='batch_{id}.hdf', additional_request=snapshot_request)
    )

    n = 10
    print("Training for", n, "iterations")

    with build(batch_provider_tree) as minibatch_maker:
        for i in range(n):
            minibatch_maker.request_batch(request)

    print("Finished")

# first test simple augmentation, try without installing gunpowder!

In [19]:
import logging
import random
import numpy as np

from .batch_filter import BatchFilter
from gunpowder.coordinate import Coordinate

logger = logging.getLogger(__name__)

ValueError: Attempted relative import in non-package

In [ ]:


class SimpleAugment(BatchFilter):
    '''Randomly mirror and transpose all :class:`Arrays<Array>` and
    :class:`Points` in a batch.
    Args:
        mirror_only (``list`` of ``int``, optional):
            If set, only mirror between the given axes. This is useful to
            exclude channels that have a set direction, like time.
        transpose_only (``list`` of ``int``, optional):
            If set, only transpose between the given axes. This is useful to
            limit the transpose to axes with the same resolution or to exclude
            non-spatial dimensions.
    '''

    def __init__(self, mirror_only=None, transpose_only=None):

        self.mirror_only = mirror_only
        self.transpose_only = transpose_only
        self.mirror_mask = None
        self.dims = None
        self.transpose_dims = None

    def setup(self):

        self.dims = self.spec.get_total_roi().dims()

        if self.mirror_only is None:
            self.mirror_mask = [ True ]*self.dims
        else:
            self.mirror_mask = [ d in self.mirror_only for d in range(self.dims) ]

        if self.transpose_only is None:
            self.transpose_dims = list(range(self.dims))
        else:
            self.transpose_dims = self.transpose_only

    def prepare(self, request):

        self.total_roi = request.get_total_roi()

        self.mirror = [
            random.randint(0,1)
            if self.mirror_mask[d] else 0
            for d in range(self.dims)
        ]

        t = list(self.transpose_dims)
        random.shuffle(t)
        self.transpose = list(range(self.dims))
        for o, n in zip(self.transpose_dims, t):
            self.transpose[o] = n

        logger.debug("mirror = " + str(self.mirror))
        logger.debug("transpose = " + str(self.transpose))

        reverse_transpose = [0]*self.dims
        for d in range(self.dims):
            reverse_transpose[self.transpose[d]] = d

        logger.debug("downstream request = " + str(request))

        self.__transpose_request(request, reverse_transpose)
        self.__mirror_request(request, self.mirror)

        logger.debug("upstream request = " + str(request))

    def process(self, batch, request):

        mirror = tuple(
                slice(None, None, -1 if m else 1)
                for m in self.mirror
        )
        # arrays
        for (array_key, array) in batch.arrays.items():

            array.data = array.data[mirror]
            if self.transpose != (0,1,2):
                array.data = array.data.transpose(self.transpose)
        # points
        total_roi_offset = self.total_roi.get_offset()
        for (points_key, points) in batch.points.items():

            for loc_id, syn_point in points.data.items():
                # mirror
                location_in_total_offset = np.asarray(syn_point.location) - total_roi_offset
                syn_point.location = np.asarray([self.total_roi.get_end()[dim] - location_in_total_offset[dim]
                                                 if m else syn_point.location[dim] for dim, m in enumerate(self.mirror)])
                # transpose
                if self.transpose != (0, 1, 2):
                    syn_point.location = np.asarray([syn_point.location[self.transpose[d]] for d in range(self.dims)])

                # due to the mirroring, points at the lower boundary of the ROI
                # could fall on the upper one, which excludes them from the ROI
                if not points.spec.roi.contains(syn_point.location):
                    del points.data[loc_id]

        # arrays & points
        for collection_type in [batch.arrays, batch.points]:
            for (type, collector) in collection_type.items():
                logger.debug("total ROI: %s"%self.total_roi)
                logger.debug("upstream %s ROI: %s"%(type, collector.spec.roi))
                self.__mirror_roi(collector.spec.roi, self.total_roi, self.mirror)
                logger.debug("mirrored %s ROI: %s"%(type,collector.spec.roi))
                self.__transpose_roi(collector.spec.roi, self.transpose)
                logger.debug("transposed %s ROI: %s"%(type,collector.spec.roi))


    def __mirror_request(self, request, mirror):

        for key, spec in request.items():
            self.__mirror_roi(spec.roi, self.total_roi, mirror)

    def __transpose_request(self, request, transpose):

        for key, spec in request.items():
            self.__transpose_roi(spec.roi, transpose)

    def __mirror_roi(self, roi, total_roi, mirror):

        total_roi_offset = total_roi.get_offset()
        total_roi_shape = total_roi.get_shape()

        roi_offset = roi.get_offset()
        roi_shape = roi.get_shape()

        roi_in_total_offset = roi_offset - total_roi_offset
        end_of_roi_in_total = roi_in_total_offset + roi_shape
        roi_in_total_offset_mirrored = total_roi_shape - end_of_roi_in_total
        roi_offset = Coordinate(
                total_roi_offset[d] + roi_in_total_offset_mirrored[d] if mirror[d] else roi_offset[d]
                for d in range(self.dims)
        )

        roi.set_offset(roi_offset)

    def __transpose_roi(self, roi, transpose):

        offset = roi.get_offset()
        shape = roi.get_shape()
        offset = tuple(offset[transpose[d]] for d in range(self.dims))
        shape = tuple(shape[transpose[d]] for d in range(self.dims))
        roi.set_offset(offset)
        roi.set_shape(shape)

## try mirror!

In [22]:
def setup(mirror_only=None,transpose_only=None,dims=3):
    if mirror_only is None:
        mirror_mask = [ True ]*dims
    else:
        mirror_mask = [ d in self.mirror_only for d in range(dims) ]

    if transpose_only is None:
        transpose_dims = list(range(dims))
    else:
        transpose_dims = transpose_only
    return mirror_mask,transpose_dims

In [25]:
mirror_mask,transpose_dims = setup(mirror_only=None,transpose_only=None,dims=3)
print (mirror_mask,transpose_dims)

[True, True, True] [0, 1, 2]


In [26]:
dims=3

In [ ]:
# random.randint generate random axis?

In [33]:
[random.randint(0,1) if mirror_mask[d] else 0 for d in range(dims)]

[0, 0, 1]

In [31]:
[mirror_mask[d] for d in range(dims)]

[True, True, True]